# Exemplo de Classificação Básico: **Classificação de imagens** de roupas na escala de cinza


---


- Exemplo adotado no curso **ABCIA**
- Código baseado em: https://www.tensorflow.org/tutorials/keras/classification

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Neste exemplo veremos como treinar um modelo de uma rede neural para classificação de imagens de roupas, como tênis e camisetas.

Usaremos o [tf.keras](https://www.tensorflow.org/guide/keras), uma API de alto-nível para construir e treinar modelos no TensorFlow.



---
[<img src="https://www.gstatic.com/devrel-devsite/prod/vde5e97689c1d94fa683b9e5392f0f6b6562f68c8b527194cc7ca91d97bde649f/tensorflow/images/lockup.svg">](http://www.tensorflow.org)


In [ ]:
#@title Importando Bibliotecas - TODO
--- TODO

import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
#@title Importando a base de dados Fashion MNIST

# A base de dados Fashion MNIST pode ser obtida via API do Keras junto ao Tensorflow
# https://www.tensorflow.org/datasets/catalog/fashion_mnist?hl=pt-br
fashion_mnist = keras.datasets.fashion_mnist

Esse exemplo usa a base de dados Fashion MNIST que contém 70,000 imagens em tons de cinza em 10 categorias. As imagens mostram artigos individuais de roupas com baixa resolução, ou seja, 28 por 28 pixels.

In [ ]:
#@title Carregando a base de dados - TODO
(train_images, train_labels), (test_images, test_labels) = TODO

Cada exemplo da base de dados possui uma das seguintes labels:
<table>
  <tr>
    <th>Label</th>
    <th>Classe</th>
  </tr>
  <tr>
    <td>0</td>
    <td>Camisetas/Top (T-shirt/top)</td>
  </tr>
  <tr>
    <td>1</td>
    <td>Calça (Trouser)</td>
  </tr>
    <tr>
    <td>2</td>
    <td>Suéter (Pullover)</td>
  </tr>
    <tr>
    <td>3</td>
    <td>Vestidos (Dress)</td>
  </tr>
    <tr>
    <td>4</td>
    <td>Casaco (Coat)</td>
  </tr>
    <tr>
    <td>5</td>
    <td>Sandálias (Sandal)</td>
  </tr>
    <tr>
    <td>6</td>
    <td>Camisas (Shirt)</td>
  </tr>
    <tr>
    <td>7</td>
    <td>Tênis (Sneaker)</td>
  </tr>
    <tr>
    <td>8</td>
    <td>Bolsa (Bag)</td>
  </tr>
    <tr>
    <td>9</td>
    <td>Botas (Ankle boot)</td>
  </tr>
</table>

TL;DR
- Cada linha é uma image separada
- A Coluna 1 é a label correspondente.
- As colunas restantes são números de pixels (784 no total).
- Cada valor é a escuridão do pixel (1 a 255)


A pós carregar a base de dados com a função `load_data()` são retornados quatro NumPy arrays:

* Os *arrays* `train_images` e `train_labels`  são o *conjunto de treinamento*— os dados do modelo usados para aprender.
* O modelo é testado com o *conjunto de teste*, os *arrays* `test_images` e `test_labels`.

As imagens são arrays  NumPy de 28x28, com os valores de pixels entre 0 to 255. As *labels* (alvo da classificação) são um  array  de inteiros, no intervalo de  0 a 9.

In [ ]:
# 60,000 imagens (no tamanho de 28x28) para treinar a nossa rede
train_images.shape

In [ ]:
# 10,000 imagens (no tamanho de 28x28) para testar a nossa rede
test_images.shape

In [ ]:
# https://github.com/zalandoresearch/fashion-mnist#labels
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
#@title Visualização da base de dados
"""
Vamos exibir as primeiras 25 imagens do conjunto de treinamento e exibir 
o nome da classe abaixo de cada imagem.
"""

plt.figure(figsize=(10,10))

for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

## Pré-processamento dos dados

Os dados precisam ser pré-processados antes de treinar a rede. Conforme inspecionado a primeira imagem do conjunto de treinamento, observa-se que os valores dos **pixels estão entre 0 e 255**. 

Não sabemos a melhor maneira (sem um estudo preliminar) de dimensionar os valores dos pixel para modelagem para a rede neural, mas sabemos que algum dimensionamento é necessário.

Um bom ponto de partida é normalizar os valores de pixel das imagens em tons de cinza, por exemplo, redimensioná-los para o intervalo `[0,1]`. Isso envolve primeiro converter o tipo de dados de inteiros não assinados em flutuantes e, em seguida, dividir os valores de pixel pelo valor máximo. É importante que o *conjunto de treinamento* e o *conjunto de teste* podem ser pré-processados do mesmo modo:

In [ ]:
# Remodelando a matriz para 4 dims para que ela possa funcionar com a API Keras
bkp_ori_shape_train = train_images.shape
bkp_ori_shape_test = test_images.shape

train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images  = test_images.reshape(test_images.shape[0], 28, 28, 1)
print("Shape train: ", train_images.shape)
print("Shape test : ", test_images.shape)
input_shape = (28, 28, 1)

# Converter o tipo de dados de inteiros não assinados em flutuantes
train_norm = train_images.astype('float32')
test_norm = test_images.astype('float32')

# Normalizando os códigos RGB dividindo-os para o valor RGB máximo - TODO
---- TODO

Para verificar que os dados estão no formato correto e que estamos prontos para construir e treinar a rede, vamos mostrar uma imagem do **conjunto de treinamento** com os valores normalizados.

In [ ]:
# Reshape para visualizar a primeira imagem
fisrt_train_norm = train_norm[0].reshape(28,28)

# Visualizando os valores de cada posição de pixel da imagem
fig = px.imshow(fisrt_train_norm, text_auto=True, color_continuous_scale='Greys')
fig.show()

## Construindo o modelo de classificação

Construir a rede neural requer configurar as camadas do modelo, e depois, compilar o modelo.

### Configurando as camadas

O principal bloco de construção da rede neural é a camada (*layer*). As camadas (*layers*) extraem representações dos dados inseridos na rede, onde essas representações são significativas para o problema à mão.

Muito do *deep learning* consiste em encadear simples camadas. Muitas camadas, como `tf.keras.layers.Dense`, tem parâmetros que são aprendidos durante o treinamento.

<center>
<img src="https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/74_blog_image_1.png">(https://www.superdatascience.com/blogs/convolutional-neural-networks-cnn-step-4-full-connection)
</center>

In [ ]:
# Modelo baseado em: 
# https://www.kaggle.com/code/rutvikdeshpande/fashion-mnist-cnn-beginner-98/notebook

model = keras.Sequential(
    [
    keras.layers.Conv2D(
        filters=8,
        kernel_size=5,
        padding='same',
        activation='relu',
        input_shape = input_shape),
    keras.layers.MaxPooling2D(
        pool_size=2, 
        strides=2), # diminuição das amostras de 28*28 para 14*14
    keras.layers.Conv2D(
        filters=16,
        kernel_size=5,
        padding='same',
        activation='relu'),
    keras.layers.MaxPooling2D(
        pool_size=2, 
        strides=2), # diminuição das amostras de 14*14 para 7*7
    # TODO
    --- TODO
    ]
)

### Explicação das camadas

A **camada da rede**, `tf.keras.layers.Flatten`, exemplo: transforma o formato da imagem de um array de imagens de duas dimensões (de 28 por 28 pixels) para um array de uma dimensão (de 28 * 28 = 784 pixels). Pense nessa camada como camadas não empilhadas de pixels de uma imagem e os enfilere. Essa camada não tem parâmetros para aprender; ela só reformata os dados.

<center>
<img src="https://www.simplilearn.com/ice9/free_resources_article_thumb/flattening.png"> 
(http://www.tensorflow.org)
</center>

Depois dos pixels serem achatados (*Flatten*), a rede consiste de uma sequência de duas camadas `tf.keras.layers.Dense` que são camadas neurais *densely connected*, ou *fully connected*:
- A **primeira camada da rede**, camada `Dense`, tem 120 nós (ou neurônios). 
- A **segunda camada da rede** camada `Dense`, tem 84 nós (ou neurônios).
- A **terceira camada da rede** camada `Dense` é uma *softmax*  de 10 nós que retorna um array de `10 probabilidades`, cuja soma resulta em 1. Cada nó contém um valor que indica a probabilidade de que aquela **imagem pertence a uma das 10 classes**`



In [ ]:
!pip install visualkeras

In [ ]:
import visualkeras
visualkeras.layered_view(model)

### Compilando o modelo

Antes do modelo estar pronto para o treinamento, é necessário algumas configurações a mais. Essas serão adicionadas no passo de *compilação*:

* *Função Loss* (`loss='sparse_categorical_crossentropy'`): essa mede quão precisa o modelo é durante o treinamento. Queremos minimizar a função para *guiar* o modelo para a direção certa.
* *Optimizer* (`optimizer='adam'`): isso é como o modelo se atualiza com base no dado que ele vê e sua função *loss*.
* *Métricas* (`metrics=['accuracy']`): usadas para monitorar os passos de treinamento e teste. O exemplo abaixo usa a *acurácia*, a fração das imagens que foram classificadas corretamente.

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Treinando o modelo

Treinar a rede neural requer os seguintes passos:

1. Alimente com os dados de treinamento, o modelo. Neste exemplo, os dados de treinamento são os arrays `train_norm` e `train_labels`.
2. O modelo aprende como associar as imagens as *labels*, uma das 10 possibilidades.
3. Perguntamos ao modelo para fazer previsões sobre o conjunto de teste — nesse exemplo, o array `test_norm`. Verificamos se as previsões combinaram com as *labels*  do array `test_labels`.

Para começar a treinar, chame o método `model.fit`— assim chamado, porque ele "encaixa" o modelo no conjunto de treinamento. 

À medida que o modelo treina, as métricas loss e acurácia são mostradas. O modelo atinge uma acurácia maior que 0.80 (ou maior que 80%) com o conjunto de treinamento.

In [ ]:
# FIT - TODO
--- TODO

## Avaliando a acurácia do modelo

Comparando como o modelo performou com o conjunto de teste - :

In [ ]:
# Curva de aprendizagem

loss_train = history_train.history['loss']
loss_val   = history_train.history['val_loss']

epochs = range(1,5)

plt.plot(epochs, loss_train, 'o', label='Training loss')
plt.plot(epochs, loss_val, 'r', label='validation loss')

plt.title('Learning Curves - Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
# Curva de aprendizagem

loss_train = history_train.history['accuracy']
loss_val   = history_train.history['val_accuracy']

epochs = range(1,5)

plt.plot(epochs, loss_train, 'o', label='Training Accuracy')
plt.plot(epochs, loss_val, 'r', label='Validation Accuracy')

plt.title('Learning Curves - Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

Acabou que o a acurácia com o conjunto de teste é um pouco menor do que a acurácia de treinamento. Essa diferença entre as duas acurácias representa um *overfitting*. Overfitting é modelo de aprendizado de máquina performou de maneira pior em um conjunto de entradas novas, e não usadas anteriormente, que usando o conjunto de treinamento.

In [ ]:
# Obter probabilidades da classificação - TODO
--- TODO

In [ ]:
#Obter valores da classificação 
correct = 0

list_pred_argmax = []
for p in predictions:
  list_pred_argmax.append(np.argmax(p))

list_pred_by_img = np.array(list_pred_argmax)
correct   = np.nonzero(list_pred_by_img == test_labels)[0]
incorrect = np.nonzero(list_pred_by_img != test_labels)[0]
print("Predições corretas  : ", len(correct))
print("Predições incorretas: ", len(incorrect))

from sklearn.metrics import classification_report

target_names = ["Class {}".format(i) for i in range(10)]
print(classification_report(test_labels, list_pred_by_img, target_names=target_names))


## Fazendo predições

Com o modelo treinado, faremos a predições de algumas imagens.

Aqui, o modelo previu que a *label* de cada imagem no conjunto de treinamento. Vamos olhar o resultado da primeira predição:

In [ ]:
predictions[0]

A predição é um array de 10 números. Eles representam a *confiança* do modelo que a imagem corresponde a cada um dos diferentes artigos de roupa. Podemos ver cada *label*  tem um maior valor de confiança. Então, o modelo é confiante de que esse imagem é uma bota (ankle boot) ou `class_names[9]`.

Vamos plotar algumas da previsão do modelo. Labels preditas corretamente são **azuis** e as predições erradas são **vermelhas**. Note que o modelo pode errar mesmo estando confiante.

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)

  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
test_norm_no_shape = test_norm.reshape(bkp_ori_shape_test)
test_norm_no_shape.shape

In [ ]:
# Colore as predições corretas de azul e as incorretas de vermelho.

num_rows = 5
num_cols = 5
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))

for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions, test_labels, test_norm_no_shape)
  
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions, test_labels)
plt.show()